# Sentiment Prediction from Movie Review (IMDB)

Goal: Predict positive or negative sentiments from movie reviews

Dataset: [IMDB Movie reviews sentiment classification](https://keras.io/datasets/)

In [37]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer

from keras.preprocessing.sequence import pad_sequences

## Load the data

In [38]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000,
                                                      skip_top=20,
                                                      maxlen=500)

print("Training reviews: {}".format(len(x_train)))
print("Testing reviews: {}".format(len(x_test)))

Training reviews: 25000
Testing reviews: 20947


## Process the data

In [39]:
# Input to vector
tokenizer = Tokenizer(num_words=10000)
x_train = tokenizer.sequences_to_matrix(x_train, mode='binary')
x_test = tokenizer.sequences_to_matrix(x_test, mode='binary')
print(x_train.shape)
print(x_test.shape)

# One-hot output
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(25000, 10000)
(20947, 10000)
(25000, 2)
(20947, 2)
